In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
years = list(range(2025,2026))

i need selenium web driver in order to properly saved the scandanavian names, if im not using selenium the names like luka doncic,it will interpret wrong by bsoup or requests

-------------------------------------
## Webscraping The MVP candidates per season

In [7]:
mvp_url = 'https://www.basketball-reference.com/awards/awards_{}.html'

service = Service(r"D:\pacmann_AIML\ml_process\MVP_Finalist_NBA\chromedriver.exe")
driver = webdriver.Chrome(service=service)

dfs = []
for year in years:
    url = mvp_url.format(year)
    driver.get(url)
    time.sleep(3)
    
    driver.execute_script("window.scrollTo(1,100000)")# i need to scroll down,if not it will only saved the first table
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    over_header = soup.find('tr', class_='over_header')
    if over_header:
        over_header.decompose() # delete the overheader if not it will saved to data
    
    mvp_table = soup.find(id='mvp')

    if mvp_table:
        mvp = pd.read_html(str(mvp_table))[0]
        mvp['year'] = year
        dfs.append(mvp)

    time.sleep(3)

driver.quit()


C:\Users\Wahyunan\AppData\Local\Temp\ipykernel_15184\2325653463.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]
C:\Users\Wahyunan\AppData\Local\Temp\ipykernel_15184\2325653463.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]


check the length of data

In [8]:
len(dfs)

2

concat all of year into one variables

In [9]:
mvps = pd.concat(dfs, ignore_index=True)

save to csv

In [ ]:
mvps.to_csv("new_mvps_ws.csv")

------
## Webscraping the player stat per season

In [148]:

player_url = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'

dataframes_player = []
for year in years:
    service = Service(r"D:\pacmann_AIML\ml_process\MVP_Finalist_NBA\chromedriver.exe")
    driver = webdriver.Chrome(service=service)
    url = player_url.format(year)
    driver.get(url)
    driver.execute_script("window.scrollTo(1,100000)")# i need to scroll down,if not it will only saved the first table
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    player_table = soup.find(id='per_game_stats')
    if player_table:
        player = pd.read_html(str(player_table))[0]
        
        player['year'] = year
        # since the max coloumn was 24, i need to exclude the next title coloumn not to saved by webdriver
        player = player[(player['Player'] != 'Player') & ~player['Player'].str.contains('League', na=False, case=False)]
        
        dataframes_player.append(player)
    
    time.sleep(3)
    driver.quit()

C:\Users\Wahyunan\AppData\Local\Temp\ipykernel_16044\49863677.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player = pd.read_html(str(player_table))[0]
C:\Users\Wahyunan\AppData\Local\Temp\ipykernel_16044\49863677.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player = pd.read_html(str(player_table))[0]


In [149]:
len(dataframes_player)

2

In [150]:
player_df = pd.concat(dataframes_player, ignore_index=True)

In [151]:
player_df.to_csv('player_1991_2024.csv', index=False)

---------------------
## Webscraping the player advanced stats per season

In [152]:
players_url = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'

dataframes = []
for year in years:
    service = Service(r"D:\pacmann_AIML\ml_process\MVP_Finalist_NBA\chromedriver.exe")
    driver = webdriver.Chrome(service=service)
    url = players_url.format(year)
    driver.get(url)
    driver.execute_script("window.scrollTo(1,100000)") # i need to scroll down,if not it will only saved the first table
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    player_table = soup.find(id='advanced')
    if player_table:
        player = pd.read_html(str(player_table))[0]
        
        player['year'] = year
        
        # since the max coloumn was 24, i need to exclude the next title coloumn not to saved by webdriver
        player = player[(player['Player'] != 'Player') & ~player['Player'].str.contains('League', na=False, case=False)]
        dataframes.append(player)
    
    time.sleep(3)
    driver.quit()


C:\Users\Wahyunan\AppData\Local\Temp\ipykernel_16044\1073913308.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player = pd.read_html(str(player_table))[0]
C:\Users\Wahyunan\AppData\Local\Temp\ipykernel_16044\1073913308.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player = pd.read_html(str(player_table))[0]


In [153]:
len(dataframes)

2

In [154]:
final_df = pd.concat(dataframes, ignore_index=True)

In [155]:
final_df.shape

(1414, 30)

In [156]:
final_df.to_csv('player_stats_1991_2024.csv', index=False)

----------------
## Webscraping the team stats

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

team_dfs = []
team_stats_url = 'https://www.basketball-reference.com/leagues/NBA_{}_standings.html'

for year in years:
    url = team_stats_url.format(year)

    data = requests.get(url)
    soup = BeautifulSoup(data.text, 'html.parser')

    # Process Eastern Conference
    team_table = soup.find(id='divs_standings_E')
    if team_table:
        team = pd.read_html(str(team_table))[0]
        team['year'] = year
        team['team'] = team['Eastern Conference']
        del team['Eastern Conference']

        # Remove rows that represent divisions, not teams
        team = team[~team['team'].str.contains('Division')]

        team_dfs.append(team)
    time.sleep(3)

    # Process Western Conference
    team_table = soup.find(id='divs_standings_W')
    if team_table:
        team = pd.read_html(str(team_table))[0]
        team['year'] = year
        team['team'] = team['Western Conference']
        del team['Western Conference']

        # Remove rows that represent divisions, not teams
        team = team[~team['team'].str.contains('Division')]

        team_dfs.append(team)
    time.sleep(3)

# Now team_dfs will contain data only for teams, without divisions.


C:\Users\Wahyunan\AppData\Local\Temp\ipykernel_15184\640507418.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  team = pd.read_html(str(team_table))[0]
C:\Users\Wahyunan\AppData\Local\Temp\ipykernel_15184\640507418.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  team = pd.read_html(str(team_table))[0]
C:\Users\Wahyunan\AppData\Local\Temp\ipykernel_15184\640507418.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  team = pd.read_html(str(team_table))[0]
C:\Users\Wahyunan\AppData\Local\Temp\ipykernel_15184\640507418.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in

In [4]:
len(team_dfs)

4

In [3]:
all_teams = pd.concat(team_dfs)

In [5]:
all_teams.to_csv('all_teams.csv')